In [91]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [92]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import warnings

In [93]:
data=pd.read_csv("co2_emissions (1).csv",sep=";")

In [94]:
data[data.duplicated()]

,make,model,vehicle_class,engine_size,cylinders,transmission,fuel_type,fuel_consumption_city,fuel_consumption_hwy,fuel_consumption_comb(l/100km),fuel_consumption_comb(mpg),co2_emissions
1075,ACURA,RDX AWD,SUV - SMALL,3.5,6,AS,Z,12.1,8.7,10.6,27,244
1076,ACURA,RLX,MID-SIZE,3.5,6,AS,Z,11.9,7.7,10.0,28,230
1081,ALFA ROMEO,4C,TWO-SEATER,1.8,4,AM,Z,9.7,6.9,8.4,34,193
1082,ASTON MARTIN,DB9,MINICOMPACT,5.9,12,A,Z,18.0,12.6,15.6,18,359
1084,ASTON MARTIN,V8 VANTAGE,TWO-SEATER,4.7,8,AM,Z,17.4,11.3,14.7,19,338
...,...,...,...,...,...,...,...,...,...,...,...,...
7356,TOYOTA,Tundra,PICKUP TRUCK - STANDARD,5.7,8,AS,X,17.7,13.6,15.9,18,371
7365,VOLKSWAGEN,Golf GTI,COMPACT,2.0,4,M,X,9.8,7.3,8.7,32,203
7366,VOLKSWAGEN,Jetta,COMPACT,1.4,4,AS,X,7.8,5.9,7.0,40,162
7367,VOLKSWAGEN,Jetta,COMPACT,1.4,4,M,X,7.9,5.9,7.0,40,163


In [95]:
data.drop_duplicates(inplace=True)

In [96]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data1 = data.copy()
data1["transmission"] = labelencoder.fit_transform(data1["transmission"])
data1["fuel_type"] = labelencoder.fit_transform(data1["fuel_type"])
data1["vehicle_class"] = labelencoder.fit_transform(data1["vehicle_class"])

In [97]:
Q1=data1.quantile(0.25)
Q3=data1.quantile(0.75)
IQR=Q3-Q1
print(IQR)

vehicle_class                      9.0
engine_size                        1.7
cylinders                          2.0
transmission                       1.0
fuel_type                          1.0
fuel_consumption_city              4.6
fuel_consumption_hwy               2.8
fuel_consumption_comb(l/100km)     3.8
fuel_consumption_comb(mpg)        10.0
co2_emissions                     81.0
dtype: float64


In [98]:
data1=data1[~((data1<(Q1-1.5*IQR))|(data1>(Q3+1.5*IQR))).any(axis=1)]
data1.shape

C:\Users\Pavani\AppData\Local\Temp/ipykernel_13504/1240933532.py:1: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version.  Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  data1=data1[~((data1<(Q1-1.5*IQR))|(data1>(Q3+1.5*IQR))).any(axis=1)]


(4454, 12)

In [99]:
data2=data1.drop("model", axis=1)

In [100]:
data3=data2.drop("make", axis=1)

In [101]:
array = data3.values
X = array[:,0:9]
Y = array[:,9]

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=10)

In [103]:
# feature extraction
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
model = LinearRegression()


rfe = RFE(model, n_features_to_select=4)
fit = rfe.fit(X, Y)

fit.n_features_
fit.support_
fit.ranking_

array([6, 2, 1, 5, 3, 1, 1, 1, 4])

In [104]:
from sklearn.ensemble import ExtraTreesClassifier
# feature extraction
model = ExtraTreesClassifier()
model.fit(X, Y)

print(model.feature_importances_)

[0.10030704 0.10023745 0.01884378 0.06636657 0.0323902  0.19182197
 0.20202073 0.20573944 0.08227283]


In [105]:
# Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model = RandomForestRegressor()

model.fit(X_train,Y_train)
model_pred=model.predict(X_test)

model_accuracy_RF=r2_score(Y_test,model_pred)
print('Random Forest: ',model_accuracy_RF)

Random Forest:  0.9950350069180628
